# ERIKS Digital - Customer Name String Extraction

1. I was asked by a member of the Marketing team, to produce a set of files of customer names, divided up by country and turnover class(importance to us). 
2. There are six classes (A-F), and two countries (Netherlands and Belgium). 
3. The turnover class is typically provided along with each order entered into the SQL database. 
4. However, the turnover class changes over time as those responsible for this measure reclassify customers over time. 
5. Thus, I was to take the turnover class upon the last order placed by each customer, as their turnover class. 
6. I then had to write each set of customer names to a .csv file, in the following format:
    a. |Customer_1|Customer_2|...|Customer_n|
7. I made a simple SQL query to extract this orders data first, then analysed this data here.

# Setup and Data Import

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import warnings
warnings.filterwarnings('ignore')
from datetime import date
import matplotlib.pyplot as plt
from matplotlib.dates import date2num
import re
import csv

In [2]:
def import_and_clean_orders_nlbe_data(filepath):
    df_orders = pd.read_csv(filepath)
    df_orders.CustomerId = df_orders.CustomerId.astype(str) 
    df_orders.TurnoverClass = df_orders.TurnoverClass.astype(str)
    df_orders.CustomerMarketPotential = df_orders.CustomerMarketPotential.astype(str)
    df_orders.CustomerId = df_orders.CustomerId.astype(str)
    df_orders['SalesDocumentHeaderCreationDate'] = pd.to_datetime(df_orders['SalesDocumentHeaderCreationDate'])
    return df_orders

In [3]:
df_orders = import_and_clean_orders_nlbe_data(r"OrdersNLBE_Condensed.csv")

# Create intermediate last order date dataframe


In [4]:
df_last_order_date = df_orders.loc[df_orders.groupby('CustomerId').SalesDocumentHeaderCreationDate.idxmax()]
df_last_order_date = df_last_order_date[['CustomerId', 'SalesDocumentHeaderCreationDate']]

# Filter original orders dataframe with last orders dataframe

In [5]:
keys = list(df_last_order_date.columns.values)
i1 = df_orders.set_index(keys).index
i2 = df_last_order_date.set_index(keys).index
df_orders_filtered = df_orders[i1.isin(i2)]
print (len(df_orders_filtered))

42193


# Check if all turnover classes unique per customer


In [6]:
unique_turnover_classes = df_orders_filtered.groupby('CustomerId')['TurnoverClass'].nunique()

In [7]:
# If this block prints nothing, all turnover classes for each customer are now unique
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    for i in unique_turnover_classes:
        if i > 1:
            print (i)

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


This means that a few customers have a changing turnover class on the day of their most recent order!

## Multiple turnover classes - what's going on?

In [8]:
unique_turnover_classes_multiple_turnover_classes = unique_turnover_classes[unique_turnover_classes > 1]
unique_turnover_classes_multiple_turnover_classes.head(n = 50)
list_customers_multiple_turnovers = list(unique_turnover_classes_multiple_turnover_classes.index)
print (list_customers_multiple_turnovers)

['1026211', '1026700', '1029555', '1031795', '1033116', '1033378', '1033481', '1034154', '1035323', '1037922', '1040276', '1041566', '1043106', '1049434', '1065965', '1717359', '1726415', '1741920', '1776097', '1781592', '1796362', '1796746', '1797447']


Email these back to Marketing team, to say that they have multiple turnover classes on the last day. 

In [9]:
df_orders_filtered_multiple_turnover_class_customers = df_orders_filtered[df_orders_filtered['CustomerId'].isin(list_customers_multiple_turnovers)]
print (len(df_orders_filtered_multiple_turnover_class_customers))

46


In [10]:
df_orders_filtered_multiple_turnover_class_customers_anon = df_orders_filtered_multiple_turnover_class_customers.drop(['CustomerName'], axis=1)

df_orders_filtered_multiple_turnover_class_customers_anon.head(n = 100)

,SalesDocumentHeaderCreationDate,CustomerMarketPotential,TurnoverClass,IndustryCode2Description,SalesOrganization,CustomerId
1776,2019-11-13,nan,B,Chemical,EB01,1049434
1779,2019-11-13,A5,nan,Chemical,EN01,1049434
3017,2019-02-26,nan,E,Other industries,EB01,1776097
3018,2019-02-26,nan,F,Other industries,EB01,1776097
3787,2019-09-06,nan,A,Food Beverage Tobacc,EB01,1717359
3788,2019-09-06,nan,B,Food Beverage Tobacc,EB01,1717359
9137,2019-03-15,nan,D,Dist. & retailers,EB01,1037922
9138,2019-03-15,nan,F,Dist. & retailers,EB01,1037922
11021,2018-12-21,nan,nan,Food and beverage,EB01,1781592
11027,2018-12-21,nan,F,Food and beverage,EB01,1781592


This is a small list of customers, so let's remove them from the main last_orders dataframe and pass them in a separate csv later. 

## Remove customers with multiple turnover classes on last order date

In [11]:
df_orders_filtered = pd.concat([df_orders_filtered, df_orders_filtered_multiple_turnover_class_customers]).drop_duplicates(keep=False)
print (len(df_orders_filtered))

42147


Length was previously 42193, we had 46 columns from customers with multiple turnover classes (on latest order day), removing them leaves 42147, looks good. 

# Remove unnecessary columns

In [12]:
df_orders_filtered = df_orders_filtered[['CustomerId', 'CustomerName', 'TurnoverClass', 'CustomerMarketPotential', 'IndustryCode2Description', 'SalesOrganization']]
df_orders_filtered.CustomerMarketPotential.value_counts()

nan    23737
E1      6180
D1      2260
Z5      1375
A9      1332
D5       953
C1       886
A5       849
B5       813
B1       648
E5       610
C5       572
E9       569
Z1       517
D9       407
B9       122
P        106
A1       100
C9        72
Z9        39
Name: CustomerMarketPotential, dtype: int64

In [13]:
df_orders_filtered.TurnoverClass.value_counts()

nan    31162
F       6713
C       1419
A        869
E        780
D        712
B        492
Name: TurnoverClass, dtype: int64

# Remove numbers from CustomerMarketPotential

In [14]:
df_orders_filtered['CustomerMarketPotential'] = df_orders_filtered['CustomerMarketPotential'].str.replace('\d+', '')
df_orders_filtered.CustomerMarketPotential.value_counts()

nan    23737
E       7359
D       3620
A       2281
Z       1931
B       1583
C       1530
P        106
Name: CustomerMarketPotential, dtype: int64

# If duplicate Turnover Class/Potential check is valid, drop all duplicate customer rows

In [15]:
df_orders_filtered = df_orders_filtered.drop_duplicates('CustomerId')
df_orders_filtered.CustomerMarketPotential.value_counts()

nan    23713
E       7345
D       3617
A       2277
Z       1912
B       1578
C       1527
P        106
Name: CustomerMarketPotential, dtype: int64

# Produce 2 lists of turnover classes, NL and BE ( not used with static list creation)


In [16]:
list_NL_Classes = df_orders_filtered.CustomerMarketPotential.value_counts().index.tolist()
list_BE_Classes = df_orders_filtered.TurnoverClass.value_counts().index.tolist()

# Filter dataframe on each turnover class, extract customer names

In [17]:
def get_lists(dataframe, turnover_class, country_code):
    if str(country_code) == 'NL':
        list_customers = dataframe[dataframe['CustomerMarketPotential'] == turnover_class].CustomerName.value_counts().index.tolist()
    elif country_code == 'BE':
        list_customers = dataframe[dataframe['TurnoverClass'] == turnover_class].CustomerName.value_counts().index.tolist()
    else:
        print ("Wrong country code, 'NL' or 'BE' only")
    list_customers.sort()
    return list_customers

In [18]:
list_Customers_NL_A = get_lists(df_orders_filtered, 'A', 'NL')
list_Customers_NL_B = get_lists(df_orders_filtered, 'B', 'NL')
list_Customers_NL_C = get_lists(df_orders_filtered, 'C', 'NL')
list_Customers_NL_D = get_lists(df_orders_filtered, 'D', 'NL')
list_Customers_NL_E = get_lists(df_orders_filtered, 'E', 'NL')
list_Customers_NL_F = get_lists(df_orders_filtered, 'F', 'NL')
#Ideally, these variables would be created dynamically, by appending each class name in above class list to a base variable name.

list_Customers_BE_A = get_lists(df_orders_filtered, 'A', 'BE')
list_Customers_BE_B = get_lists(df_orders_filtered, 'B', 'BE')
list_Customers_BE_C = get_lists(df_orders_filtered, 'C', 'BE')
list_Customers_BE_D = get_lists(df_orders_filtered, 'D', 'BE')
list_Customers_BE_E = get_lists(df_orders_filtered, 'E', 'BE')
list_Customers_BE_F = get_lists(df_orders_filtered, 'F', 'BE')


# Clean strings, keeping only letters and numbers

In [19]:
list_of_lists = [list_Customers_NL_A, list_Customers_NL_B, list_Customers_NL_C, list_Customers_NL_D, list_Customers_NL_E, list_Customers_NL_F, 
                list_Customers_BE_A, list_Customers_BE_B, list_Customers_BE_C, list_Customers_BE_D, list_Customers_BE_E, list_Customers_BE_F]

for list_instance in list_of_lists:
    for index, i in enumerate(list_instance):
        i = re.sub('[^a-zA-Z0-9]+', ' ', i)
        list_instance[index] = i

In [20]:
def turn_list_into_bar_delimited_string(filename, list_of_customers):
    with open(filename, 'w') as out_f:
        for index, l in enumerate(list_of_customers):
            if index == len(list_of_customers) - 1:          
                out_f.write(l)
            else:
                out_f.write(l + '|')

# Write results to csv files

In [21]:
#NL
turn_list_into_bar_delimited_string(r'NL_A.csv', list_Customers_NL_A)
turn_list_into_bar_delimited_string(r'NL_B.csv', list_Customers_NL_B)
turn_list_into_bar_delimited_string(r'NL_C.csv', list_Customers_NL_C)
turn_list_into_bar_delimited_string(r'NL_D.csv', list_Customers_NL_D)
turn_list_into_bar_delimited_string(r'NL_E.csv', list_Customers_NL_E)
turn_list_into_bar_delimited_string(r'NL_F.csv', list_Customers_NL_F)

turn_list_into_bar_delimited_string(r'BE_A.csv', list_Customers_BE_A)
turn_list_into_bar_delimited_string(r'BE_B.csv', list_Customers_BE_B)
turn_list_into_bar_delimited_string(r'BE_C.csv', list_Customers_BE_C)
turn_list_into_bar_delimited_string(r'BE_D.csv', list_Customers_BE_D)
turn_list_into_bar_delimited_string(r'BE_E.csv', list_Customers_BE_E)
turn_list_into_bar_delimited_string(r'BE_F.csv', list_Customers_BE_F)
